# Notebook to evaluate a trained model with pyfolio
Note: pyfolio must be install using pip install git+https://github.com/quantopian/pyfolio. There are known bugs with pip install pyfolio.

In [ ]:
import os
import glob

import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter

from deep_rl_asset_allocation.configs import paths_config, data_config
from deep_rl_asset_allocation.utils import data_loader_utils
import deep_rl_asset_allocation.preprocessing.data_preprocessing as data_preprocessing

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# plotting in notebook
%matplotlib inline

### Helper Functions

In [ ]:
def get_daily_return_as_df(df):
    df['daily_return'] = df.account_value.pct_change(1)
    return df


def get_daily_return_as_series(df):
    df1 = df.copy()
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1.set_index('Date', drop=False, inplace=True)
    df1.index = df1.index.tz_localize('UTC')
    del df1['Date']
    ts = pd.Series(df1['daily_return'].values, index=df1.index)
    return ts

def get_training_results_account_value(df_unique_trade_dates, results_csv_dir=paths_config.results_csv_dir):
    df_account_value = pd.DataFrame()
    filepath = os.path.join(results_csv_dir, "test", f'portfolio_value_*.csv')
    num_csv_results_files = len(glob.glob(filepath))
    for idx in range(0, num_csv_results_files):
        csv_results_file = os.path.join(results_csv_dir, "test", f'portfolio_value_{idx}.csv')
        print(f"csv_results_file: {csv_results_file}")
        df = pd.read_csv(csv_results_file)
        df_account_value = df_account_value.append(df, ignore_index=True)
    # change the column name
    df_account_value = pd.DataFrame({'account_value': df_account_value['0']})
    # merge with unique trade dates from 2016-01-04 or index 64
    df_account_value = df_account_value.join(df_unique_trade_dates.reset_index(drop=True))
    df_account_value = df_account_value[['Date', 'account_value']]
    return df_account_value

### Benchmark: DJIA Index

In [ ]:
# load DJIA - out of sample data (test data)
df_djia_out_of_sample = pd.read_csv(paths_config.TESTING_DATA_FILE)
df_djia_out_of_sample["Date"] = df_djia_out_of_sample.apply(data_preprocessing.convert_datadate_to_datetime, axis=1)

# get data between dates
df_djia_out_of_sample = df_djia_out_of_sample[(df_djia_out_of_sample['Date'] >= data_config.TESTING_START) & (df_djia_out_of_sample['Date'] <= data_config.TESTING_END)]
df_djia_out_of_sample = df_djia_out_of_sample.reset_index(drop=True)

In [ ]:
# get daily return as relative difference in adjusted close price of DJIA
df_djia_out_of_sample['daily_return'] = df_djia_out_of_sample['Adj Close'].pct_change(1)
df_djia_out_of_sample = df_djia_out_of_sample.reset_index(drop=True).dropna()
# df_djia_out_of_sample

In [ ]:
series_djia_out_of_sample = get_daily_return_as_series(df_djia_out_of_sample)
# series_djia_out_of_sample

### Benchmark: Min-variance

In [ ]:
# read pkl file
df_min_variance = pd.read_pickle('zipline/benchmark_min_var.pkl').reset_index(drop=True)
df_min_variance["Date"] = df_min_variance["period_open"]
df_min_variance["Date"] = df_min_variance.apply(data_preprocessing.convert_datadate_to_datetime, axis=1)
# df_min_variance

### RL: Training Results

In [ ]:
# load Dow 30 - in sample data (training data)
df_djia_in_sample = data_loader_utils.load_preprocessed_djia_data()

# rebalance_window is the number of months to retrain the model
rebalance_window = data_config.REBALANCE_WINDOW
# validation_window is the number of months to validation the model and select for trading
validation_window = data_config.VALIDATION_WINDOW

# get unique trade dates to load model results
unique_trade_dates = data_loader_utils.get_data_between_dates(df_djia_in_sample, start=data_config.VALIDATION_START, end=data_config.TESTING_END).Date.unique()
df_unique_trade_dates = pd.DataFrame({'Date': unique_trade_dates})

# filter for when Testing Starts
df_unique_trade_dates = df_unique_trade_dates[df_unique_trade_dates.Date >= data_config.TESTING_START]
df_unique_trade_dates.index = df_unique_trade_dates.Date.factorize()[0]
# df_unique_trade_dates

In [ ]:
results_dir = os.path.join(paths_config.parent_dir, 'results')
results_csv_dir = os.path.join(results_dir, 'csv')

df_training_results_account_value = get_training_results_account_value(df_unique_trade_dates, results_csv_dir)

In [ ]:
# get daily return as relative difference in account value
df_training_results_account_value['daily_return'] = df_training_results_account_value.account_value.pct_change(1)

# compare dates with test date
df_training_results_account_value['Date'] = df_djia_out_of_sample['Date']
df_training_results_account_value = df_training_results_account_value[['Date', 'account_value', 'daily_return']]

df_training_results_account_value = df_training_results_account_value.reset_index(drop=True).dropna()
# df_training_results_account_value

In [ ]:
series_training_results_account_value = get_daily_return_as_series(df_training_results_account_value)  # [0:1097]
# series_training_results_account_value

### Plot results

In [ ]:
# init the plot
fig, ax = plt.subplots(figsize=(20, 6))
plt.style.use("fivethirtyeight")

ax.set(xlabel="Trading Dates",
       ylabel="Cumulative Return [%]",
       title="Cumulative Return with Transaction Costs",
       )

# init plot
# ax.plot(df_dates.index, df_dates.zeroes, alpha=0.01)

# plot actual data
ax.plot(df_djia_out_of_sample["Date"], df_djia_out_of_sample["daily_return"].cumsum(), label="DJIA")
# ax.plot(df_min_variance["Date"], df_min_variance["returns"].cumsum(), label="Min-variance")
ax.plot(df_training_results_account_value["Date"], df_training_results_account_value["daily_return"].cumsum(), label="A2C")

# format plot
date_form = DateFormatter("%Y")
ax.xaxis.set_major_formatter(date_form)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=12))

# ax.set_xlim([pd.Timestamp(year=2015, month=6, day=1), pd.Timestamp(year=2021, month=1, day=1)])
plt.legend(loc='upper left')